# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 29 2022 8:17AM,243037,10,SO94335,"Senseonics, Incorporated",Released
1,Jul 29 2022 8:16AM,243036,20,ALUR798918905,Alumier Labs Inc.,Released
2,Jul 29 2022 8:16AM,243036,20,ALUR172858180,Alumier Labs Inc.,Released
3,Jul 29 2022 8:16AM,243036,20,ALUR173560548,Alumier Labs Inc.,Released
4,Jul 29 2022 8:16AM,243036,20,ALUR126414729,Alumier Labs Inc.,Released
5,Jul 29 2022 8:16AM,243036,20,ALUR320017708,Alumier Labs Inc.,Released
6,Jul 29 2022 8:16AM,243033,20,ALUR740277910,Alumier Labs Inc.,Released
7,Jul 29 2022 8:03AM,243035,10,NST-1093,"Amcyte Pharma, Inc.",Released
8,Jul 29 2022 8:01AM,243034,10,Enova-10383,Emerginnova,Released
9,Jul 29 2022 8:01AM,243034,10,Eme-102319,Emerginnova,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
18,243033,Released,1
19,243034,Released,2
20,243035,Released,1
21,243036,Released,5
22,243037,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
243033,NaN,NaN,1.0
243034,NaN,NaN,2.0
243035,NaN,NaN,1.0
243036,NaN,NaN,5.0
243037,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242914,0.0,0.0,1.0
242937,0.0,5.0,6.0
242964,17.0,9.0,8.0
242965,0.0,0.0,1.0
242967,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242914,0,0,1
242937,0,5,6
242964,17,9,8
242965,0,0,1
242967,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242914,0,0,1
1,242937,0,5,6
2,242964,17,9,8
3,242965,0,0,1
4,242967,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242914,,,1
1,242937,,5,6
2,242964,17,9,8
3,242965,,,1
4,242967,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 29 2022 8:17AM,243037,10,"Senseonics, Incorporated"
1,Jul 29 2022 8:16AM,243036,20,Alumier Labs Inc.
6,Jul 29 2022 8:16AM,243033,20,Alumier Labs Inc.
7,Jul 29 2022 8:03AM,243035,10,"Amcyte Pharma, Inc."
8,Jul 29 2022 8:01AM,243034,10,Emerginnova
10,Jul 28 2022 3:44PM,243024,12,Proven Skincare
11,Jul 28 2022 3:41PM,243019,20,Alumier Labs Inc.
30,Jul 28 2022 3:30PM,243021,16,"SHL Pharma, LLC"
31,Jul 28 2022 3:12PM,243017,22,"NBTY Global, Inc."
32,Jul 28 2022 3:10PM,243015,22,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 29 2022 8:17AM,243037,10,"Senseonics, Incorporated",,,1
1,Jul 29 2022 8:16AM,243036,20,Alumier Labs Inc.,,,5
2,Jul 29 2022 8:16AM,243033,20,Alumier Labs Inc.,,,1
3,Jul 29 2022 8:03AM,243035,10,"Amcyte Pharma, Inc.",,,1
4,Jul 29 2022 8:01AM,243034,10,Emerginnova,,,2
5,Jul 28 2022 3:44PM,243024,12,Proven Skincare,,1,
6,Jul 28 2022 3:41PM,243019,20,Alumier Labs Inc.,,,19
7,Jul 28 2022 3:30PM,243021,16,"SHL Pharma, LLC",,1,
8,Jul 28 2022 3:12PM,243017,22,"NBTY Global, Inc.",,,1
9,Jul 28 2022 3:10PM,243015,22,"NBTY Global, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 29 2022 8:17AM,243037,10,"Senseonics, Incorporated",1,,
1,Jul 29 2022 8:16AM,243036,20,Alumier Labs Inc.,5,,
2,Jul 29 2022 8:16AM,243033,20,Alumier Labs Inc.,1,,
3,Jul 29 2022 8:03AM,243035,10,"Amcyte Pharma, Inc.",1,,
4,Jul 29 2022 8:01AM,243034,10,Emerginnova,2,,
5,Jul 28 2022 3:44PM,243024,12,Proven Skincare,,1,
6,Jul 28 2022 3:41PM,243019,20,Alumier Labs Inc.,19,,
7,Jul 28 2022 3:30PM,243021,16,"SHL Pharma, LLC",,1,
8,Jul 28 2022 3:12PM,243017,22,"NBTY Global, Inc.",1,,
9,Jul 28 2022 3:10PM,243015,22,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 29 2022 8:17AM,243037,10,"Senseonics, Incorporated",1,,
1,Jul 29 2022 8:16AM,243036,20,Alumier Labs Inc.,5,,
2,Jul 29 2022 8:16AM,243033,20,Alumier Labs Inc.,1,,
3,Jul 29 2022 8:03AM,243035,10,"Amcyte Pharma, Inc.",1,,
4,Jul 29 2022 8:01AM,243034,10,Emerginnova,2,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 29 2022 8:17AM,243037,10,"Senseonics, Incorporated",1.0,NaN,NaN
1,Jul 29 2022 8:16AM,243036,20,Alumier Labs Inc.,5.0,NaN,NaN
2,Jul 29 2022 8:16AM,243033,20,Alumier Labs Inc.,1.0,NaN,NaN
3,Jul 29 2022 8:03AM,243035,10,"Amcyte Pharma, Inc.",1.0,NaN,NaN
4,Jul 29 2022 8:01AM,243034,10,Emerginnova,2.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 29 2022 8:17AM,243037,10,"Senseonics, Incorporated",1.0,0.0,0.0
1,Jul 29 2022 8:16AM,243036,20,Alumier Labs Inc.,5.0,0.0,0.0
2,Jul 29 2022 8:16AM,243033,20,Alumier Labs Inc.,1.0,0.0,0.0
3,Jul 29 2022 8:03AM,243035,10,"Amcyte Pharma, Inc.",1.0,0.0,0.0
4,Jul 29 2022 8:01AM,243034,10,Emerginnova,2.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1214957,11.0,5.0,0.0
12,243024,0.0,1.0,0.0
16,971956,3.0,1.0,0.0
20,972052,33.0,9.0,17.0
21,729019,3.0,0.0,0.0
22,729008,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1214957,11.0,5.0,0.0
1,12,243024,0.0,1.0,0.0
2,16,971956,3.0,1.0,0.0
3,20,972052,33.0,9.0,17.0
4,21,729019,3.0,0.0,0.0
5,22,729008,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,11.0,5.0,0.0
1,12,0.0,1.0,0.0
2,16,3.0,1.0,0.0
3,20,33.0,9.0,17.0
4,21,3.0,0.0,0.0
5,22,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,11.0
1,12,Released,0.0
2,16,Released,3.0
3,20,Released,33.0
4,21,Released,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,20,21,22
Status,,,,,,
Completed,0.0,0.0,0.0,17.0,0.0,0.0
Executing,5.0,1.0,1.0,9.0,0.0,0.0
Released,11.0,0.0,3.0,33.0,3.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,20,21,22
0,Completed,0.0,0.0,0.0,17.0,0.0,0.0
1,Executing,5.0,1.0,1.0,9.0,0.0,0.0
2,Released,11.0,0.0,3.0,33.0,3.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,20,21,22
0,Completed,0.0,0.0,0.0,17.0,0.0,0.0
1,Executing,5.0,1.0,1.0,9.0,0.0,0.0
2,Released,11.0,0.0,3.0,33.0,3.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()